In [153]:
from googletrans import Translator
import speech_recognition as sr
from gtts import gTTS
import os
import pyaudio
import simpleaudio as sa
import numpy as np
import _pickle as cPickle
from scipy.io.wavfile import read
import python_speech_features as MFCC
from sklearn import preprocessing
import warnings
import pickle
import pandas as pd
from scipy import stats

warnings.filterwarnings("ignore")

In [157]:
r = sr.Recognizer()

with sr.Microphone() as source:
    print('SAY SOMETHING...')
    audio = r.listen(source)
    print('TIME OVER, THANKS!')

with open("microphone-results.wav", "wb") as f:
    f.write(audio.get_wav_data())

SAY SOMETHING...
TIME OVER, THANKS!


In [158]:
def get_MFCC(sr,audio):
    features = MFCC.mfcc(audio,sr,0.025,0.01,13,appendEnergy=False)
    feat = np.asarray(())

    for i in range(features.shape[0]):
        temp = features[i,:]
        if np.isnan(np.min(temp)):
            continue
        else:
            if feat.size == 0 :
                feat = temp
            else:
                feat = np.vstack((feat,temp))
    
    features = feat
    features = preprocessing.scale(features)
    return features                

def gmm_model():
    sourcePath = "/Users/jashrathod/PycharmProjects/On-the-go-Translator/" 
    modelPath = "/Users/jashrathod/PycharmProjects/On-the-go-Translator/Voice-Based-Gender-Detection/Modules/"

    gmmFiles = [os.path.join(modelPath,fname) for fname in os.listdir(modelPath) if fname.endswith('.gmm')]
    models = [cPickle.load(open(fname,'rb')) for fname in gmmFiles]
    # print(models)

    genders = [fname.split("/")[-1].split(".gmm")[0] for fname in gmmFiles]
    files = [os.path.join(sourcePath,f) for f in os.listdir(sourcePath) if f.endswith(".wav")]

    # for f in files:

    f = sourcePath + "microphone-results.wav"
#     print(f.split("/")[-1])
    sr,audio = read(f)
    features = get_MFCC(sr,audio)
    scores = None
    log_likelihood = np.zeros(len(models))

    for i in range(len(models)):
        gmm = models[i]
        scores = np.array(gmm.score(features))
        log_likelihood[i] = scores.sum()
       
    winner = np.argmax(log_likelihood)
    winner = (winner+1)%2
    print(genders[winner])

gmm_model()

female


In [159]:
try:
    text_audio = r.recognize_google(audio)
    print("text: ", text_audio)
except:
    pass

text:  I am going home


In [160]:
translator = Translator()

dt1 = translator.detect(text_audio)
print(dt1)

Detected(lang=en, confidence=1.0)


In [161]:
l = ["hi", "gu", "fr"]

In [162]:
for i in l:
    translated = translator.translate(text_audio, dest=i)
    a = translated.text

    speech = gTTS(text=a, lang=i, slow=False)

    speech.save("text-" + i + ".wav")
#     os.system("start text-" + i + ".mp3")

In [163]:
# import parselmouth
# from parselmouth.praat import call

# sound = parselmouth.Sound("text-hi.wav")

In [164]:
# manipulation = call(sound, "To Manipulation", 0.001, 75, 600)

In [165]:
# pitch_tier = call(manipulation, "Extract pitch tier")

# call(pitch_tier, "Multiply frequencies", sound.xmin, sound.xmax, 1)
# # print(sound.xmin, sound.xmax)

# call([pitch_tier, manipulation], "Replace pitch tier")
# sound_octave_up = call(manipulation, "Get resynthesis (overlap-add)")

In [168]:
# from IPython.display import Audio
# Audio(data=sound_octave_up.values, rate=sound_octave_up.sampling_frequency)